## Nonbinary classification neural network from scratch

In [109]:
import os
import numpy as np
import torch
import requests
import tarfile
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import time

from PIL import Image
from torchvision import transforms
from torch import tensor
from torch.utils.data import DataLoader
from pathlib import Path
from dataclasses import dataclass
from fastbook import show_image

#### MetaParameters
- keeping track of all global variables: Batch size, learning rate, number of epochs

In [2]:
### ADJUST METAPARAMETERS HERE
@dataclass
class MetaParameters:
    batch_size: int
    learning_rate: float
    n_epochs: int
    
mp = MetaParameters(64, 1., 8)
mp

MetaParameters(batch_size=64, learning_rate=1.0, n_epochs=8)

In [22]:
cuda = torch.device('cuda')
cpu = torch.device('cpu')

In [21]:
def ls(path: Path):
    return [p for p in path.glob('./*')]

In [113]:
path = Path('/home/c/projects/NN-from-scratch')
dls = {
    'train': DataLoader(datasets.MNIST(path, download=True, train=True, transform=transforms.ToTensor()),
                        batch_size=mp.batch_size,
                        shuffle=True),
    'valid': DataLoader(datasets.MNIST(path, download=True, train=False, transform=transforms.ToTensor()),
                        batch_size=mp.batch_size,
                        shuffle=True),
}
#https://blog.paperspace.com/dataloaders-abstractions-pytorch/

In [117]:
batch = next(iter(dls['train']))
batch[1].shape

torch.Size([64])

In [ ]:
def model(x, w, b):
    return x@w + b

In [65]:
def mnist_loss(predictions, targets) -> float:
    return -(predictions.softmax(-1)[targets].log())

In [91]:
preds = torch.rand(10) * 10
targets = torch.zeros(10)
targets[3] = 1
for x, y in dls['train']:
    print(x, y)
    break

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [23]:
def init_params(shape):
    return torch.randn(shape, requires_grad=True, dtype=torch.float, device=cuda)

In [ ]:
def train(dl, params):
    for x, y in dl:
        preds = model(x, weights, bias)
        loss = mnist_loss(preds, y)
        loss.backward()
        for p in params:
            p.data -= p.grad * mp.learning_rate
            p.grad.zero_()

In [107]:
weights = init_params((28*28, 1))
bias = init_params(1)
preds = (torch.rand(10, 10)).softmax(-1)
targs = (torch.rand(10)).softmax(-1)
(preds.argmax(1)==targs).float()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [108]:
def batch_accuracy(preds, trgts):
    preds = preds.softmax(-1)
    return ((preds.argmax(1)) == trgts).float().mean()

def accuracy(valid_dl, weights, bias):
    accs = [batch_accuracy(model(xb, weights, bias), yb) for xb, yb in valid_dl]
    return f'{round(torch.stack(accs).mean().item(), 3) * 100}%'

In [ ]:
params = weights, bias
for i in range(mp.n_epochs):
    #train_epoch(train_dl, model, mp.learning_rate, params)
    #print(validate_epoch(model))
    train(dls['train'], params)
    print(accuracy(dls['valid'], weights, bias))

0.9541667
0.962882
0.9662269
0.9690509
0.969132
0.9709144
0.9735186
0.9740394
